# Tensorflow Validation with Tensorflow Pipeline

We have seen how to generate baseline statistics and schema, and now we need to include the data validation as part of our ML workflow. Tensorflow Pipeline provides components to let you easily do that, and this is what we are going to do now. 

Specifically, we will build a data pipeline by:

- Extracting data from BigQuery using Tensorflow Pipeline.
- Generating the statistics of the dataset.
- Creating a schema as per the domain knowledge.
- Apply schema to a new dataset.
- Check for anomalies.

### Dataset

This notebook uses the same dataset as the previous lab: Chicago crime data.

[the previous lab](https://www.freeldom.com/article/TFDV%20-Lab-1-1622385519244)

This notebook uses [Chicago crime data](https://data.cityofchicago.org/) published as a public dataset in BigQuery. This dataset reflects reported incidents of crime that occurred in the City of Chicago from 2001 to the present, minus the most recent seven days. The data will be extracted with the following columns:

- **date**: Date when the incident occurred.
- **iucr**: The Illinois Unifrom Crime Reporting code.
- **primary_type**: The primary description of the IUCR code.
- **location_description**: Description of the location where the incident occurred.
- **arrest**: Indicates whether an arrest was made.
- **domestic**: Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence Act.
- **district**: Indicates the police district where the incident occurred. 
- **ward**: The ward (City Council district) where the incident occurred.
- **fbi_code**: Indicates the crime classification.
- **year**: Year the incident occurred.

### Installing dependencies

In [ ]:
!pip install tfx

### Imports

In [39]:
import tensorflow as tf
import tfx

import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.extensions.google_cloud_big_query.example_gen.component import BigQueryExampleGen
from tensorflow_metadata.proto.v0 import schema_pb2

from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import ImporterNode

from tfx.types import standard_artifacts

from datetime import datetime

CHICAGO_CRIME_TABLE = 'bigquery-public-data.chicago_crime.crime'

GCS_BUCKET = "" # Set your GCS bucket
PROJECT_ID = '' # Set your GCP Project Id
REGION = '' # Set the region for Dataflow jobs

#PIPELINE_ID = datetime.now().strftime('%Y%m%d-%H%M%S')

### Setup Interactive Context

We will set up the Interactive Context so we can manually execute the pipeline components from the notebook.

In [4]:
PIPELINE_ROOT = './pipeline'

# Declare the InteractiveContext and use a local sqlite file as the metadata store.
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Extracting data from BigQuery

The first step of the pipeline is to extract the data from BigQuery. In the previous lab, we did it using our own DataFlow pipeline. This time we will use the tfx component [BigQueryExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/extensions/google_cloud_big_query/example_gen/component/BigQueryExampleGen), but the query remains the same.


In [6]:
def generate_query(year_from= None, year_to= None, limit= None) -> str:
    query = f"""
        SELECT 
            FORMAT_DATE('%Y',  CAST(date AS DATE)) AS crime_year,
            FORMAT_DATE('%b',  CAST(date AS DATE)) AS crime_month,
            FORMAT_DATE('%d',  CAST(date AS DATE)) AS crime_day, 
            FORMAT_DATE('%a',  CAST(date AS DATE)) AS crime_day_of_week, 
            iucr,
            primary_type,
            location_description,
            CAST(domestic AS INT64) AS domestic,
            district,
            ward,
            fbi_code,
            CAST(arrest AS INT64) AS arrest,
        FROM 
          {CHICAGO_CRIME_TABLE}
        """
    if year_from:
        query += f"WHERE year >= {year_from}"
        if year_to:
            query += f" AND year <= {year_to} \n"
    if limit:
        query  += f"LIMIT {limit}"
        
    return query

In [17]:
# Some parameters needed for BigQuery, but remember we run the pipeline locally, so we will extract only 2019 data
example_config = [
    '--project', PROJECT_ID,
    '--temp_location', f"gs://{GCS_BUCKET}/tfdv/pipeline/{PIPELINE_ID}/",
    '--region', REGION
]

example_gen = BigQueryExampleGen(query=generate_query(2019, 2019))
context.run(example_gen, beam_pipeline_args=example_config)


ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 6
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 5
        type_id: 5
        uri: "./pipeline/BigQueryExampleGen/examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "None"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1622368417218
        last_update_time_since_epoch: 1622368543162
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Computing statistics

Next, let's compute the statistics to observe and analyze characteristics of our data. We will use the component [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) to do that. We need to feed in the data we have extracted previously. We use the component outputs that has the path of where the data has been extracted.

In [20]:
statistics_gen = StatisticsGen(
      examples=example_gen.outputs['examples']
)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 7
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 6
        type_id: 7
        uri: "./pipeline/StatisticsGen/statistics/7"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [21]:
# Let's show the statistics
context.show(statistics_gen.outputs['statistics'])

## Inferring the schema

As usual, we need to create a schema that we will use to validate incoming datasets during training and serving. The TFX component to do that is named [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen) component.

In [22]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics']
)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 8
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 7
        type_id: 9
        uri: "./pipeline/SchemaGen/schema/8"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [23]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'arrest',INT,required,,-
'crime_day',BYTES,required,,-
'crime_day_of_week',STRING,required,,'crime_day_of_week'
'crime_month',STRING,required,,'crime_month'
'crime_year',BYTES,required,,-
'district',INT,required,,-
'domestic',INT,required,,-
'fbi_code',STRING,required,,'fbi_code'
'iucr',BYTES,required,,-


/Users/matthieu/dev/freeldom/mlops-framework/venv/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'crime_day_of_week',"'Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed'"
'crime_month',"'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep'"
'fbi_code',"'01A', '01B', '02', '03', '04A', '04B', '05', '06', '07', '08A', '08B', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '22', '24', '26'"
'primary_type',"'ARSON', 'ASSAULT', 'BATTERY', 'BURGLARY', 'CONCEALED CARRY LICENSE VIOLATION', 'CRIM SEXUAL ASSAULT', 'CRIMINAL DAMAGE', 'CRIMINAL SEXUAL ASSAULT', 'CRIMINAL TRESPASS', 'DECEPTIVE PRACTICE', 'GAMBLING', 'HOMICIDE', 'HUMAN TRAFFICKING', 'INTERFERENCE WITH PUBLIC OFFICER', 'INTIMIDATION', 'KIDNAPPING', 'LIQUOR LAW VIOLATION', 'MOTOR VEHICLE THEFT', 'NARCOTICS', 'NON-CRIMINAL', 'OBSCENITY', 'OFFENSE INVOLVING CHILDREN', 'OTHER NARCOTIC VIOLATION', 'OTHER OFFENSE', 'PROSTITUTION', 'PUBLIC INDECENCY', 'PUBLIC PEACE VIOLATION', 'ROBBERY', 'SEX OFFENSE', 'STALKING', 'THEFT', 'WEAPONS VIOLATION'"


We also know that we must try to enhance the inferred schema with the domain knowledge that we have. Let's apply the same constraints as the ones of the first lab. First, we get the path where our schema has been saved, and load it.

In [32]:
schema_uri = schema_gen.outputs['schema']._artifacts[0].uri
baseline_schema = tfdv.load_schema_text(schema_uri + '/schema.pbtxt')

# Setting domain
tfdv.set_domain(baseline_schema, "domestic", schema_pb2.IntDomain(min=0, max=1))
tfdv.set_domain(baseline_schema, "district", schema_pb2.IntDomain(min=1, max=31))

# Adding crime type
primary_types = tfdv.get_domain(baseline_schema, 'primary_type')
primary_types.value.append('RITUALISM')

# Setting up environments
baseline_schema.default_environment.append('TRAINING')
baseline_schema.default_environment.append('SERVING')
tfdv.get_feature(baseline_schema, 'arrest').not_in_environment.append('SERVING')

In [33]:
tfdv.display_schema(schema=baseline_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'arrest',INT,required,,-
'crime_day',BYTES,required,,-
'crime_day_of_week',STRING,required,,'crime_day_of_week'
'crime_month',STRING,required,,'crime_month'
'crime_year',BYTES,required,,-
'district',INT,required,,min: 1; max: 31
'domestic',INT,required,,min: 0; max: 1
'fbi_code',STRING,required,,'fbi_code'
'iucr',BYTES,required,,-


/Users/matthieu/dev/freeldom/mlops-framework/venv/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'crime_day_of_week',"'Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed'"
'crime_month',"'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep'"
'fbi_code',"'01A', '01B', '02', '03', '04A', '04B', '05', '06', '07', '08A', '08B', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '22', '24', '26'"
'primary_type',"'ARSON', 'ASSAULT', 'BATTERY', 'BURGLARY', 'CONCEALED CARRY LICENSE VIOLATION', 'CRIM SEXUAL ASSAULT', 'CRIMINAL DAMAGE', 'CRIMINAL SEXUAL ASSAULT', 'CRIMINAL TRESPASS', 'DECEPTIVE PRACTICE', 'GAMBLING', 'HOMICIDE', 'HUMAN TRAFFICKING', 'INTERFERENCE WITH PUBLIC OFFICER', 'INTIMIDATION', 'KIDNAPPING', 'LIQUOR LAW VIOLATION', 'MOTOR VEHICLE THEFT', 'NARCOTICS', 'NON-CRIMINAL', 'OBSCENITY', 'OFFENSE INVOLVING CHILDREN', 'OTHER NARCOTIC VIOLATION', 'OTHER OFFENSE', 'PROSTITUTION', 'PUBLIC INDECENCY', 'PUBLIC PEACE VIOLATION', 'ROBBERY', 'SEX OFFENSE', 'STALKING', 'THEFT', 'WEAPONS VIOLATION', 'RITUALISM'"


In [46]:
# Let's save the updated schema
schema_folder = f'{PIPELINE_ROOT}/enhanced_schema'
!mkdir -p {schema_folder}
schema_filename = f'{schema_folder}/schema.pbtxt'

# Save the curated schema to the said file
tfdv.write_schema_text(baseline_schema, schema_filename)

## Building the pipeline

We have our baseline schema and statistics, and the goal is to use them for our next training job or our next serving job. In the next cells, we are going to use TFX components to import our baseline statistics, extract new dataset, compute the statistics and compare with our baseline. In a real-world situation, if we identify anomalies, the ML pipeline should raise an alert. Otherwise, we would continue with preprocessing, training and serving. 

### Import new schema

We use [ImporterNode](https://www.tensorflow.org/tfx/guide/statsgen#using_the_statsgen_component_with_a_schema) to import the schema.

In [48]:
imported_baseline_schema = ImporterNode(
    source_uri=schema_folder,
    artifact_type=standard_artifacts.Schema
).with_id('baseline_schema')
context.run(imported_baseline_schema, enable_cache=False)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 14
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 12
        type_id: 7
        uri: "./pipeline/StatisticsGen/statistics/14"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## Extract data from 2020

Let's extract 2020 data and compute the statistics. We will then check for anomalies using our baseline schema and [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) component.

In [49]:
# Some parameters needed for BigQuery, but remember we run the pipeline locally, so we will extract only 2019 data
example_config = [
    '--project', PROJECT_ID,
    '--temp_location', f"gs://{GCS_BUCKET}/tfdv/pipeline/{PIPELINE_ID}/",
    '--region', REGION
]

example_gen = BigQueryExampleGen(query=generate_query(2020, 2020))
context.run(example_gen, beam_pipeline_args=example_config)


/Users/matthieu/dev/freeldom/mlops-framework/venv/lib/python3.8/site-packages/apache_beam/io/gcp/bigquery.py:1936: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(


ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 15
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 13
        type_id: 5
        uri: "./pipeline/BigQueryExampleGen/examples/15"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "None"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [50]:
statistics_gen = StatisticsGen(
      examples=example_gen.outputs['examples']
)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 16
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 14
        type_id: 7
        uri: "./pipeline/StatisticsGen/statistics/16"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [54]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=imported_baseline_schema.outputs['result']
)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 17
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 15
        type_id: 12
        uri: "./pipeline/ExampleValidator/anomalies/17"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.30.0"
          }
        }
        state: LIVE
        , artifact_type: id: 12
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [55]:
# Show anomalies
context.show(example_validator.outputs['anomalies'])

/Users/matthieu/dev/freeldom/mlops-framework/venv/lib/python3.8/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


No anomalies! We could continue to the next steps of our pipeline!

## End of lab

The lab ends here as we don't intend to do any preprocessing or model training. We have seen how to use TFX components to include data validation as part of your ML workflow and import a baseline schema. It's always good to check for anomalies in your input data, but you can also do it after your preprocessing steps to ensure nothing wrong happened.